In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LogisticRegression, SGDClassifier
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, accuracy_score
from scipy.sparse import csr_matrix

In [4]:
dtypes = {'nt_fp': str, 'event_date': str, 's_datefrom': str,
          's_price': float, 's_currency': str, 's_country': str, 
          'day_of_week': str, 'a_month': str, 'hour_of_day': int,
          's_depcity, ': str, 's_adults': int, 's_children': int,
          'ua_browser': str, 'ua_device_type': str, 'ua_platform': str,
          'ip_lat': int, 'ip_long': int}

parse_dates = ['event_date', 's_datefrom']

data_df = pd.read_csv('~/Desktop/events-by-5-month.csv', index_col=False,  dtype=dtypes, parse_dates=parse_dates)

In [5]:
k = {'BYN': 31., 'UAH': 2.4, 'USD': 62., 'RUB': 1., 'EUR': 72., 'KZT': 0.19}


In [3]:
# вычисляем "глубину" поиска
data_df['deep'] = ((data_df['s_datefrom'] - data_df['event_date'])).dt.days
# вычисляем часть дня
data_df['part_of_day'] = data_df['hour_of_day']//3
# отделяем цель
y =  data_df['s_price'].values
# отделяем признаки
data_features = data_df.drop(columns=['s_price', 'event_date', 'nt_fp', 's_datefrom', 'hour_of_day'])
# заменяем категориальные признаки количественными (0 или 1)
data_dummies = pd.get_dummies(data_features)
C = data_dummies.values
# создаём разреженную матрицу (без нулей). 
# На практике так не делают, т.к. неразреженная матрица 'а' может не поместиться в памяти
X = csr_matrix(C)
# разделяем признаки и цели на учебные и тестовые
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=17)

In [4]:
10**6/X_train.shape[0]

22.605000226050002

In [5]:
# обучаем логистическую регрессию
logit = LogisticRegression(solver='lbfgs', random_state=17, n_jobs=-1)
# обучаем стохастический градиентный спуск
# max_iter - максимальное число проходов по всей выборке. Определяется, как 10**6, делённое на размерность выборки
sgd_logit = SGDClassifier(max_iter=22,random_state=17, n_jobs=-1)

In [ ]:
%%time
logit.fit(X_train, y_train)

In [ ]:
%%time
sgd_logit.fit(X_train, y_train)

In [ ]:
# смотрим аккуратность логистической регрессии
accuracy_score(y_test, logit.predict(X_test))

In [ ]:
# смотрим аккуратность стохастического градиентного спуска
accuracy_score(y_test, sgd_logit.predict(X_test))